In [4]:
#projet UVSQolor le but du projet consiste a cree un logiciel dans lequel on peut manipuler des images ce programme devra : 

# permettre d'ouvrir une image 
# proposer plusieurs filtre et algorithme de modification 
# permettre de sauvegarder une image 
#gerer un historique d'action


#une image avec plusieurs filtres les filtres que nous devrons coder seront :


#filtre vert rouge bleu rgb 
#Filtre gris 
#Luminositer (correction gamma)
#contraste  (correction sigma)
#flou uniforme
#flou gaussien
#filtre de detection de bord 
#fusion d'image

#autre choix (zoom d'image par ex,redimensionner l'image par exemple, rotate ect )




# nous crerons l'interface graphique avec tkinter le but sera de cree un canva dans lequel l'utilisateur qui lance le code pourra modifier n'importe quel image 
#cela suppose que nous devons pas importer notre image mais nous devons avoir acces a la bibliotheque personel de l'utilisateur comme on a fait 
#pour le projet 2048 pour charger le fichier que nous avons enregistrer (voir la syntaxe dans le projet 2048) on crera une fonction #charger l'image pour cette etape 

# ne supprimer pas les instructions en # elles nous seront utile pour la suite et aussi pour avoir un apercu de ce que nous devons faire 






#DEBUT DU CODE DU PROJET 




# Bibliotheque utile 

import numpy as np
import tkinter as tk
from tkinter import filedialog 
from PIL import Image
from PIL import ImageTk 
import pathlib
from scipy.signal import convolve2d

#interface

racine = tk.Tk()
racine.title('UVSQolor')

canvas = tk.Canvas(racine, width=950, height=800, bg="#333533")
canvas.grid(row=0, column=0)

#Code pour ouvrir une image un fichier en jpeg ou png 
# ON CREE LES MENUS A L'AVANCE POUR LES APPELER DANS NOS FONCTIONS PLUS TARD 

menu_principal = tk.Menu(racine)                                  #on cree un menu principal dans lequel on va stocker toute les cascade
menu_fichier = tk.Menu(menu_principal, tearoff=0)


menu_principal.add_cascade(label="Fichier", menu=menu_fichier)           # on cree un espace fichier 
racine.config(menu=menu_principal)



# ESPACE MODIFIER OU IL VA Y AVOIR LES DIFFERENTS FILTRES 
# C'EST ICI QU'ON CREE TOUT LES CALLBACKS DANS LE MENU PRINCIPAL 

menu_edit = tk.Menu(menu_principal, tearoff=0)                              # autre menu modifier on pourra mettre les filtres dedans 
menu_principal.add_cascade(label="Modifier", menu=menu_edit) 

menu_filtre = tk.Menu(menu_principal, tearoff=0)                              # autre menu modifier on pourra mettre les filtres dedans 
menu_principal.add_cascade(label="Filtre", menu=menu_filtre) 









# fonction pour ouvrir un fichier image les extensions jpeg png etc ....

def open_fichier():
    global image_loader, image

    # Ouvre une boîte de dialogue pour choisir une image
    fichier_path = filedialog.askopenfilename(
        defaultextension=".png",
        filetypes=[("Images", "*.png *.jpeg *.jpg")])

    if fichier_path:
        image_loader = Image.open(fichier_path).convert('RGB')
        image_loader = image_loader.resize((475, 300))
        image_canva(canvas, racine)

menu_fichier.add_command(label="Open", command=open_fichier)

# enregistrer une image 

def save_image():                                      
    global image, uvsq_root , canvas_used

    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("Images", "*.png *.jpeg *.jpg")])
    if file_path:
        with open(file_path, 'w') as file:                              #enregistre le fichier dans la bibliotheque de l'utilisateur 
            file.write(image)
menu_fichier.add_command(label="Enregistrer", command=save_image) # cascade dans le callback du menu principal fichier  

#fonction pour fermer la fenetre racine 

def quitter_racine():                  
    racine.destroy()
menu_fichier.add_command(label="Quitter", command=quitter_racine) # sous callback du menu principal fichier 

# ESPACE OU L'ON PEUT METTRE LES BOUTONS                                                 
frame_controle = tk.Frame(racine)
frame_controle.grid(row=0, column=1) 

# Bouton RETOUR EN ARRIERE 

def bouton_arriere(): 
    global image 
    
# A COMPLETER POUR L'HISTORIQUE DES TOUTCHES 


arriere = tk.Button(frame_controle, text="<-----", font=("Helvetica", 20, "bold"), command=bouton_arriere)  # vérifie a chaque fin de déplacement si on a pas une fin possible
arriere.grid(row=1, column=0, padx=5)

# BOUTON POUR REVENIR EN AVANT SI ON SOUHAITE REVENIR SUR NOTRE IDEE DE BASE PAR EX 
 
def bouton_avant(): 
    global image 
 # A COMPLETER POUR L'HISTORIQUE DES MODIFICATIONS 

avant = tk.Button(frame_controle, text="----->", font=("Helvetica", 20, "bold"), command=bouton_avant)  # vérifie a chaque fin de déplacement si on a pas une fin possible
avant.grid(row=1, column=2, padx=5)


# FIN DE L'INTERFACE  










# ON DOIT DEFINIR LES PIXELS RGB        # ON DOIT REVOIR CAR C'EST TROP CHATGPT JE CROIS 
def rgb_pixel(pixel):
	r,g,b = pixel
	return (r+g+b)//3


def average_pix(pixel):
	r,g,b = pixel
	return (r+g+b)//3


# FONCTION 
def get_pixel_rgb(x, y,image = ''):
	if image == '':
		image=image_loader
	return image.getpixel((x, y))

def image_path(image_name):
	return f'{pathlib.Path(__file__).parent.resolve()}/{image_name}'

global image_history_filters , image_history_redo, image_save
image_history_filters = []
image_history_redo = [] #si on veut remettre la photo comme elle etait avant le Redo
canvas_created = False
image_save = ''

def add_history(image):
	image_history_filters.append(np.array(image))
      
def image_canva(background_canva , uvsq_root):
	global image
	image = ImageTk.PhotoImage(image_loader)
	def canvas_create_funct(uvsq_root):
		print('working')
		global canvas_created, image_canvas , canvas_used
		canvas_used = background_canva
		canvas_created = True
		image_canvas = background_canva.create_image(0, 0,
							image = image,
				         anchor = "nw")
	if canvas_created:
		background_canva.itemconfig(image_canvas  , image=image)
	else:
		canvas_create_funct(uvsq_root)


def apply_filter(background_canva , uvsq_root, image_name = 'modified_image.jpg' , image_redo = False):
	global image , image_loader , image_history_redo
	image_loader = Image.open(image_path(image_name))
	add_history(np.array(image_loader.convert("RGB")))
	if not image_redo:
		image_history_redo = []
	image_canva(background_canva , uvsq_root)


















# FILTRE BLEU 


def blue_filter():
    global image, image_loader

    # On part de image_loader, qui est bien une image PIL
    pixels = np.array(image_loader.convert('RGB'))

    for y in range(pixels.shape[0]):
        for x in range(pixels.shape[1]):
            _, _, b = pixels[y, x]
            pixels[y, x] = (0, 0, b)

    # On crée une nouvelle image depuis les pixels
    image_loader = Image.fromarray(pixels)

    # On met à jour l’historique si tu veux gérer undo/redo
    add_history(image_loader)

    # On affiche l’image modifiée dans le canvas
    image_canva(canvas, racine)
menu_filtre.add_command(label="filtre bleu", command=blue_filter)



 



















#def Luminositer(): 
    #global image                                    # CODE DU COURS DANS LE TD 6 VOIR LA QUESTION 3 LE CODE EST DEJA DONNER 

    #global dialogue_effet
    #dialogue_effet = tk.Toplevel(racine)
    #dialogue_effet.title("Luminosité")
    #dialogue_effet.geometry("300x150")
    #dialogue_effet.grab_set()
    #slider = tk.Scale(dialogue_effet, from_=0.1, to=3.0,
     #                 orient=tk.HORIZONTAL, length=200,
      #                resolution=0.1, digits=2,
       #               command=correction_gamma)
    #slider.set(1.0)
    #slider.pack(pady=20)

    #frame_boutons = tk.Frame(dialogue_effet)
    #frame_boutons.pack(side=tk.BOTTOM, pady=10)

    #bouton_appliquer = tk.Button(frame_boutons, text="Appliquer",
                                 #command=applique_effet)
    #bouton_appliquer.pack(side=tk.LEFT, padx=10)

    #bouton_annuler = tk.Button(frame_boutons, text="Annuler",
                               #command=annule_effet)
    #bouton_annuler.pack(side=tk.LEFT, padx=10)

#menu_edit.add_command(label="Luminositer", command=Luminositer)
























def contraste(): 
    global image 

# a completer 

menu_edit.add_command(label="contraste", command=contraste)





def flou_uniforme():
    global image 
# a completer 

menu_edit.add_command(label="flou uniforme", command=flou_uniforme)

































# AIDE quand on decoule la cascade il y a l'entete help document pour afficher le 2e canvas 


menu_aide = tk.Menu(menu_principal, tearoff=0)                  
menu_principal.add_cascade(label="Aide", menu=menu_aide)           


def fichier_help():          # ici on a cree une autre fenetre canvas par dessus le canvas de base pour expliquer les differentes fonctionnaliter des filtres par ex 
    global aide              

    aide = tk.Canvas(racine, width=950, height=800, bg='white')
    aide.grid(row=0, column=0)

    # Texte et bouton
    aide.create_text(350, 260, text="ici on explique les filtres ect pour aider l'utilisateur", font=("Helvetica", 10, "bold"), fill='black')
    aide.create_window(350, 320, window=tk.Menu(aide, text="ici on explique les filtres ect pour aider l'utilisateur", font=("Helvetica", 10, "bold"), command=fichier_help))

menu_aide.add_command(label="Help document", command=fichier_help)







racine.mainloop()

working
